In [1]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv1D,Input,MaxPooling1D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [2]:
cats=os.listdir("F:/desktop2/CNN Models/categories/MV")

In [3]:
cats

['Absent', 'Present']

In [4]:
from tqdm.notebook import tqdm

In [14]:
all_wavs=[]
all_label=[]

for cat in cats:
    print(cat)
    wavs=os.listdir(f"F:/desktop2/CNN Models/categories/MV/{cat}")
    for wav in tqdm(wavs):
        samples,sample_rate=librosa.load(f"F:/desktop2/CNN Models/categories/MV/{cat}/{wav}",sr=8000)
        mfcc=librosa.feature.mfcc(y=samples, sr=sample_rate, n_mfcc=13, n_fft=2048, hop_length=512 )
        mfcc = mfcc.T
        all_wavs.append(np.asarray(mfcc))
        all_label.append(cat)
       

Absent


  0%|          | 0/1062 [00:00<?, ?it/s]

Present


  0%|          | 0/460 [00:00<?, ?it/s]

In [15]:
len(all_wavs),len(all_label)

(1522, 1522)

In [16]:
all_wavs[0].shape

(357, 13)

In [17]:
all_wavs[1].shape

(357, 13)

In [18]:
import pandas as pd
df=pd.DataFrame({"audio":all_wavs,"labels":all_label})

In [19]:
present=df[df["labels"] == "Present"]
present.shape

(460, 2)

In [20]:
absent=df[df["labels"] == "Absent"]
absent.shape

(1062, 2)

In [21]:
absent_sample=absent.sample(600)
absent_sample.shape

(600, 2)

In [22]:
df_balanced=pd.concat([present,absent_sample])

In [23]:
df_balanced.head()

,audio,labels
1062,"[[-86.879036, 189.13715, -62.138206, 18.788097...",Present
1063,"[[-56.4342, 150.32954, -36.054157, 9.015192, 2...",Present
1064,"[[-47.6196, 140.78622, -31.478783, 7.7257195, ...",Present
1065,"[[-91.377304, 193.72762, -63.128143, 17.282349...",Present
1066,"[[-86.24824, 187.18828, -58.121544, 14.670029,...",Present


In [24]:
wavs=df_balanced["audio"]

In [25]:
wavs

1062    [[-86.879036, 189.13715, -62.138206, 18.788097...
1063    [[-56.4342, 150.32954, -36.054157, 9.015192, 2...
1064    [[-47.6196, 140.78622, -31.478783, 7.7257195, ...
1065    [[-91.377304, 193.72762, -63.128143, 17.282349...
1066    [[-86.24824, 187.18828, -58.121544, 14.670029,...
                              ...                        
647     [[-130.32277, 99.53124, 12.465538, -21.820126,...
126     [[-134.18301, 105.97238, -17.429684, 6.0610447...
504     [[-136.64511, 80.054276, 16.510471, -10.865601...
541     [[-132.52116, 133.90971, -33.2358, -22.212513,...
44      [[-72.50276, 107.67087, -18.356138, -10.751805...
Name: audio, Length: 1060, dtype: object

In [26]:
all_wavs[2].shape

(357, 13)

In [27]:
labels=df_balanced["labels"]

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
X = np.asarray(wavs)
y = np.asarray(labels)

In [30]:
X = tf.keras.preprocessing.sequence.pad_sequences(X)
X.shape

(1060, 967, 13)

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
le=LabelEncoder()
y=le.fit_transform(labels)

In [33]:
y[:25]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [34]:
y.shape

(1060,)

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [37]:
X_train.shape,X_test.shape

((954, 967, 13), (106, 967, 13))

In [38]:
y_train

array([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [39]:
X_train_cnn = X_train.reshape(X_train.shape[0], 967, 13, 1)
X_test_cnn = X_test.reshape(X_test.shape[0], 967, 13, 1)

In [40]:
X_train_cnn.shape

(954, 967, 13, 1)

In [41]:
## RNN Model
def build_model(input_shape):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(tf.keras.layers.LSTM(64))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(1, activation='sigmoid'))

    return model

In [42]:
def get_model():
    model = Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(967,13 , 1)))
    model.add(tf.keras.layers.Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(tf.keras.layers.Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    return model

In [43]:
# create network
model = build_model((967,13))

# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 967, 128)          72704     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6

In [44]:
model.fit(X_train_cnn,y_train,epochs=100,verbose=1)

Epoch 1/100
30/30 [==============================] - 29s 800ms/step - loss: 0.6476 - accuracy: 0.6164
Epoch 2/100
30/30 [==============================] - 24s 786ms/step - loss: 0.6062 - accuracy: 0.6918
Epoch 3/100
30/30 [==============================] - 24s 797ms/step - loss: 0.5541 - accuracy: 0.7579
Epoch 4/100
30/30 [==============================] - 24s 798ms/step - loss: 0.4929 - accuracy: 0.7862
Epoch 5/100
30/30 [==============================] - 24s 807ms/step - loss: 0.4608 - accuracy: 0.7956
Epoch 6/100
30/30 [==============================] - 24s 801ms/step - loss: 0.4158 - accuracy: 0.8218
Epoch 7/100
30/30 [==============================] - 24s 794ms/step - loss: 0.3549 - accuracy: 0.8501
Epoch 8/100
30/30 [==============================] - 24s 790ms/step - loss: 0.3358 - accuracy: 0.8449
Epoch 9/100
30/30 [==============================] - 24s 791ms/step - loss: 0.3354 - accuracy: 0.8574
Epoch 10/100
30/30 [==============================] - 24s 793ms/step - loss: 0.279

In [45]:
# create network
model_cnn = get_model()

# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model_cnn.compile(optimizer="RMSprop",
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 966, 12, 32)       160       
                                                                 
 conv2d_1 (Conv2D)           (None, 965, 11, 48)       6192      
                                                                 
 conv2d_2 (Conv2D)           (None, 964, 10, 120)      23160     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 482, 5, 120)      0         
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 482, 5, 120)       0         
                                                                 
 flatten (Flatten)           (None, 289200)            0         
                                                      

In [46]:
model_cnn.fit(X_train,y_train,epochs=100,verbose=1)

Epoch 1/100
30/30 [==============================] - 32s 1s/step - loss: 18.6895 - accuracy: 0.5964
Epoch 2/100
30/30 [==============================] - 30s 1s/step - loss: 0.5027 - accuracy: 0.7642
Epoch 3/100
30/30 [==============================] - 29s 982ms/step - loss: 0.3071 - accuracy: 0.8857
Epoch 4/100
30/30 [==============================] - 29s 981ms/step - loss: 0.2227 - accuracy: 0.9319
Epoch 5/100
30/30 [==============================] - 30s 984ms/step - loss: 0.1256 - accuracy: 0.9602
Epoch 6/100
30/30 [==============================] - 29s 979ms/step - loss: 0.0665 - accuracy: 0.9801
Epoch 7/100
30/30 [==============================] - 29s 977ms/step - loss: 0.0630 - accuracy: 0.9832
Epoch 8/100
30/30 [==============================] - 29s 981ms/step - loss: 0.0432 - accuracy: 0.9885
Epoch 9/100
30/30 [==============================] - 29s 979ms/step - loss: 0.0091 - accuracy: 0.9979
Epoch 10/100
30/30 [==============================] - 30s 984ms/step - loss: 0.1811 - a

In [47]:
model.evaluate(X_test,y_test)

4/4 [==============================] - 2s 203ms/step - loss: 0.6344 - accuracy: 0.9151


[0.6343750953674316, 0.9150943160057068]

In [48]:
model_cnn.evaluate(X_test,y_test)

4/4 [==============================] - 1s 121ms/step - loss: 1.3776 - accuracy: 0.9717


[1.377643346786499, 0.9716981053352356]

In [49]:
pred=model_cnn.predict(X_test)
# pred

4/4 [==============================] - 1s 122ms/step


In [50]:
pred[1]

array([0.], dtype=float32)

In [51]:
preds=[round(p[0]) for p in pred]

In [52]:
from sklearn.metrics import classification_report

In [53]:
# model CNN result MV Model
print(classification_report(np.array(preds),y_test))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        63
           1       1.00      0.93      0.96        43

    accuracy                           0.97       106
   macro avg       0.98      0.97      0.97       106
weighted avg       0.97      0.97      0.97       106



In [54]:
# model RNN result
pred=model.predict(X_test)
preds=[round(p[0]) for p in pred]
print(classification_report(np.array(preds),y_test))

4/4 [==============================] - 2s 266ms/step
              precision    recall  f1-score   support

           0       0.92      0.94      0.93        65
           1       0.90      0.88      0.89        41

    accuracy                           0.92       106
   macro avg       0.91      0.91      0.91       106
weighted avg       0.91      0.92      0.91       106

